In [1]:
# Import libraries
import pandas as pd
import numpy as np
import pyodbc
from datetime import datetime

# Import scripts
import querying
import create_result_df
# from create_result_df import create_result_df,remove_exceptions

### CONNECTION STRING ###
# remove details prior to upload. Add details before use
cnxn = pyodbc.connect(
###

# Create a list of clinical trial objects, and add them to a list
class clinical_trial:
    def __init__(self,studynum,visits,cohort,specimen_types,exceptions):
        self.studynum = studynum
        self.visits = visits
        self.cohort = cohort
        self.specimen_types = specimen_types
        self.exceptions = exceptions

reboot = clinical_trial("ITN080AI", #studynum
                    {'A':['0A', '1A', '2A', '3A', '4A', '7A', '10A', '13A', '17A', '18A', '19A', '20A', '21A', '22A', '23A', '24A', '25A','DVA'],
                    "B":['0B', '1B', '3B', '5B', '7B', '9B', '10B', '11B', '12B', '13B', '14B', '15B', '16B', '17B','DVB'],
                    }, # Visit dictionary
                    ["A","B"], # cohort
                    {"Serum Clot":["H"+"%02d" % i for i in list(range(1,24))],
                  "PBMC":['10A','10B','10C','10D','10E','10F','10G','10H','10I'],
                  "Whole Blood\nTranscriptomics":['9A','9B'],
                  "Whole Blood\nEpigenetic":['51','52','53','54'],
                "Urine Supernatant":["D"+"%02d" % i for i in list(range(1,11))],
                "Urine Pellet":['R01','R02']
                  }, # spectype dict
                    {"A":[('PBMC','1A'),('PBMC','2A'),('PBMC','3A'),
                          ('Urine Pellet','1A'),('Urine Pellet','2A'),('Urine Pellet','3A'),
                          ('Urine Supernatant','1A'),('Urine Supernatant','2A'),('Urine Supernatant','3A'),
                          ('Whole Blood\nEpigenetic','1A'),('Whole Blood\nEpigenetic','2A'),('Whole Blood\nEpigenetic','3A'),
                          ('Whole Blood\nTranscriptomics','1A'),('Whole Blood\nTranscriptomics','2A'),('Whole Blood\nTranscriptomics','3A')],
                      "B":[]},# exceptions
                    ) 

reveal = clinical_trial("ITN086AI", # Studynum
                    ['0','3','6','8','9','10','WD'], # visits
                    None, # cohort?
                    {"Skin Biopsy":["M01","M02"],
                    "Whole Blood RNA":['9A','9B','9C'],
                    "Whole Blood DNA":['51','52','53','54','55','56'],
                    "PBMC":["10A","10B","10C","10D","10E","10F"],
                    "Serum":['H01', 'H02', 'H03', 'H04', 'H05', 'H06', 'H07', 'H08', 'H09', 'H10']}, # spec type dict
                     [('Skin Biopsy','3'),('Skin Biopsy','9')] # exceptions
                    )

graduate = clinical_trial("ITN084AD", # Studynum
                    ['-2','-1','0',"S1","S3","S4",'S5', 'S6','S8', 'S9','S10', 'S11',"S13",'S14','S15'], # visits
                    None, # cohorts?
                    #{'Nasal Brushing':0,'Nasal Fluid':0,'PBMC-Li Hep':0, 'Plasma-Li Hep':0, 'Serum-Clot':0,  'Whole Blood':0},
                      #GRADUATE specimen types granular enough. values not needed
                      ['Nasal Brushing','Nasal Fluid','PBMC-Li Hep', 'Plasma-Li Hep', 'Serum-Clot',  'Whole Blood'],
                    [("PBMC-Li Hep","-1"),("Plasma-Li Hep","-1"),
                            ("Nasal Brushing","S1"),("PBMC-Li Hep","S1"),("Plasma-Li Hep","S1"),("Whole Blood","S1"),
                            ("PBMC-Li Hep","S4"),("Plasma-Li Hep","S4"),
                            ("Nasal Brushing","S6"),("PBMC-Li Hep","S6"),("Plasma-Li Hep","S6"),("Whole Blood","S6"),
                            ("PBMC-Li Hep","S9"),("Plasma-Li Hep","S9"),
                            ("Nasal Brushing","S11"),("PBMC-Li Hep","S11"),("Plasma-Li Hep","S11"),("Whole Blood","S11"),
                            ("PBMC-Li Hep","S14"),("Plasma-Li Hep","S14"),
                            ] # exceptions
                    )
## no unexpected

teach = clinical_trial("ITN062ST",
                       {"Donor Visits":['199B','D101','D102','D103','ETERM'],
                        "Recipient Visits":['101','102','106','107','109','110','111','112','113','114','115','116','117','118','119','199A','201','221','301','306','408','410','412']},
                        # ['101','102','106','107','109','110','111','112','113','114','115','116','117','118','119','120','121','122','123','124','125','126','127','128','129','130',
                        #                     '131','199A','201','221','301','306','408','410','412','505','FCB1','FCB2','FCB3','FCB4','FCB5','FCB6','FCB7','FCB8','FCB9','701','702','703','704',
                        #                     '705','706','707','708','709','710','711','712','799A']},
                        ["Donor Visits","Recipient Visits"],
                        #{'Kidney Biopsy':0,'Serum-None':0,'Stool':0, 'Urine Pellet':0, 'Urine Super':0,  'Whole Blood':0},
                        ['Kidney Biopsy','Serum-None','Stool', 'Urine Pellet', 'Urine Super',  'Whole Blood'],
                        {"Donor Visits":[("Serum-None","D101"),("Urine Pellet","D101"),("Urine Super","D101"),("Stool","D101"),("Kidney Biopsy","D101"),
                                         ("Serum-None","D103"),("Urine Pellet","D103"),("Urine Super","D103"),("Stool","D103"),("Kidney Biopsy","D103"),
                                         ("Serum-None","D102"),("Urine Pellet","D102"),("Urine Super","D102"),("Whole Blood","D102"),("Kidney Biopsy","D102")],
                         "Recipient Visits":[("Urine Pellet","101"),("Urine Super","101"),("Stool","101"),("Kidney Biopsy","101"),
                                             ("Serum-None","106"),("Urine Pellet","106"),("Urine Super","106"),("Kidney Biopsy","106"),
                                             ("Serum-None","112"),("Urine Pellet","112"),("Urine Super","112"),("Stool","112"),("Kidney Biopsy","112"),
                                             ("Serum-None","118"),("Urine Pellet","118"),("Urine Super","118"),("Stool","118"),("Kidney Biopsy","118"),
                                             ("Serum-None","107"),("Urine Pellet","107"),("Urine Super","107"),("Stool","107"),("Kidney Biopsy","107"),
                                             ("Serum-None","110"),("Urine Pellet","110"),("Urine Super","110"),("Stool","110"),("Kidney Biopsy","110"),
                                             ("Serum-None","111"),("Urine Pellet","111"),("Urine Super","111"),("Stool","111"),("Kidney Biopsy","111"),
                                             ("Serum-None","114"),("Urine Pellet","114"),("Urine Super","114"),("Stool","114"),("Kidney Biopsy","114"),
                                             ("Serum-None","115"),("Urine Pellet","115"),("Urine Super","115"),("Stool","115"),("Kidney Biopsy","115"),
                                             ("Serum-None","117"),("Urine Pellet","117"),("Urine Super","117"),("Stool","117"),("Kidney Biopsy","117"),
                                             ("Serum-None","109"),("Urine Pellet","109"),("Urine Super","109"),("Stool","109"),("Kidney Biopsy","109"),
                                             ("Serum-None","113"),("Urine Pellet","113"),("Urine Super","113"),("Stool","113"),("Kidney Biopsy","113"),
                                             ("Serum-None","116"),("Urine Pellet","116"),("Urine Super","116"),("Stool","116"),("Kidney Biopsy","116")
                                             ]}
                        )

beat_ms = clinical_trial("ITN077AI",
                         ['0','L1','1','2','4','6','8','10','12','PT'],
                         None,
                         {"Whole Blood\nPBMC":['10A','10B','10C','10D','10E','10F','10G','10H','10I','10J'],
                          "Serum":["H"+"%02d" % i for i in list(range(1,21))],
                          "Whole Blood\nDNA Isolation":['51','52','53','54','55','56'],
                          "Whole Blood\nGene Expression":["9A","9B"],
                          "CSF Transfix":["21A"],
                          "CSF Super":["H"+"%02d" % i for i in list(range(1,49))],
                          "CSF Pellet":["J49"],
                          "Stem Cells":["SC01","SC02"]},
                          [("Stem Cells","0"),
                           ("CSF Transfix","1"),("CSF Super","1"),("CSF Pellet","1"),("Stem Cells","1"),
                           ("CSF Transfix","L1"),("CSF Super","L1"),("CSF Pellet","L1"),("Whole Blood\nPBMC","L1"),("Serum","L1"),("Whole Blood\nDNA Isolation","L1"),("Whole Blood\nGene Expression","L1"),
                           ("CSF Transfix","2"),("CSF Super","2"),("CSF Pellet","2"),("Stem Cells","2"),
                           ("CSF Transfix","4"),("CSF Super","4"),("CSF Pellet","4"),("Stem Cells","4"),
                           ("Stem Cells","6"),
                           ("CSF Transfix","8"),("CSF Super","8"),("CSF Pellet","8"),("Stem Cells","8"),
                           ("CSF Transfix","10"),("CSF Super","10"),("CSF Pellet","10"),("Stem Cells","10"),
                           ("Stem Cells","12"),
                           ("Stem Cells","PT")]
                         )

adapt = clinical_trial(studynum = "ITN089ST",
                       visits = {"Donor":['T0','103','104','105','202','203','204','205','206'],
                                 "Recipient":['-2','-1','0','8','9','14_1','15','16','17','18','22','25','26','T0','T4','T14','103','104','105','202','203','204','205','206'],
                                 "Ancillary":['A2']},
                        cohort = ["Donor","Recipient","Ancillary"],
                        specimen_types = ['Serum-None','Whole Blood','Bone Marrow','Lymph Node','Spleen'],
                        exceptions = {"Donor":[("Serum-None","T0"),("Bone Marrow","T0"),
                                               ("Lymph Node","103"),("Bone Marrow","103"),("Spleen","103"),
                                               ("Lymph Node","104"),("Bone Marrow","104"),("Spleen","104"),
                                               ("Lymph Node","105"),("Bone Marrow","105"),("Spleen","105"),
                                               ("Spleen","202"),("Lymph Node","202"),("Bone Marrow","202"),("Whole Blood","202"),
                                               ("Lymph Node","203"),("Spleen","203"),
                                               ("Lymph Node","204"),("Bone Marrow","204"),("Spleen","204"),
                                               ("Spleen","205"),("Lymph Node","205"),("Bone Marrow","205"),("Whole Blood","205"),
                                               ("Lymph Node","206"),("Bone Marrow","206"),("Spleen","206")],
                                      "Recipient":[("Lymph Node","-2"),("Bone Marrow","-2"),("Spleen","-2"),
                                                   ("Lymph Node","-1"),("Bone Marrow","-1"),("Spleen","-1"),
                                                   ("Lymph Node","0"),("Spleen","0"),
                                                   ("Lymph Node","8"),("Bone Marrow","8"),("Spleen","8"),
                                                   ("Spleen","9"),("Lymph Node","9"),("Bone Marrow","9"),("Whole Blood","9"),
                                                   ("Spleen","15"),("Lymph Node","15"),("Bone Marrow","15"),("Whole Blood","15"),
                                                   ("Lymph Node","16"),("Spleen","16"),
                                                   ("Lymph Node","17"),("Bone Marrow","17"),("Spleen","17"),
                                                   ("Spleen","22"),("Lymph Node","22"),("Bone Marrow","22"),("Whole Blood","22"),
                                                   ("Lymph Node","25"),("Bone Marrow","25"),("Spleen","25"),
                                                   ("Lymph Node","26"),("Bone Marrow","26"),("Spleen","26"),
                                                   ("Spleen","T0"),
                                                   ("Lymph Node","T4"),("Bone Marrow","T4"),("Spleen","T4"),
                                                   ("Spleen","T14"),("Lymph Node","T14"),("Bone Marrow","T14"),("Whole Blood","T14"),
                                                   ("Lymph Node","103"),("Bone Marrow","103"),("Spleen","103"),
                                                   ("Lymph Node","104"),("Bone Marrow","104"),("Spleen","104"),
                                                   ("Lymph Node","105"),("Bone Marrow","105"),("Spleen","105"),
                                                   ("Spleen","202"),("Lymph Node","202"),("Bone Marrow","202"),("Whole Blood","202"),
                                                   ("Lymph Node","203"),("Spleen","203"),
                                                   ("Lymph Node","204"),("Bone Marrow","204"),("Spleen","204"),
                                                   ("Spleen","205"),("Lymph Node","205"),("Bone Marrow","205"),("Whole Blood","205"),
                                                   ("Lymph Node","206"),("Bone Marrow","206"),("Spleen","206")],
                                        "Ancillary":[("Serum-None","A2"),("Bone Marrow","A2"),("Spleen","A2")]}
                       )

# littmus_mgh = clinical_trial("ITN073ST",
#                               {"Donor":["DD0","LD0"],
#                                "Recipient":['-1','0','101','101R','102','102R','T-5','T-5R','T0','T1','T7','T14','T28','T84','RFU','300','400','401','402','403','500','501','502','FCB1','FCB2','FCB3',
#                                              'FCB4','FCB5','FCB6','FCB7','FCB8','FCB']},
#                             ["Donor","Recipient"],
                            
                            
                            
#                             )



#2 adapt = 

#3 attain = 

## tides = 

## designate = 

########ust1d2 = 
 

#1 dare_aps = 

### cab_101 = 

### activate = 

#no sunbeam = 

clinical_trials = [reboot,reveal,graduate,teach,beat_ms,adapt]

In [2]:
output = pd.DataFrame(columns = ["Study","Cohort","Visit Number","Visit Ordinal","DaysPostScreening","Sample Type","Number at least 1 collected","Number of recorded visits","Percent"])
for trial in clinical_trials:
    if type(trial.cohort) == list:
        for chrt in trial.cohort:
            print("Starting {} Cohort {}".format(trial.studynum,chrt))

            lv_data = querying.get_lv_data(pd,cnxn,trial,chrt)

            rho_data = querying.get_rho_data(pd,cnxn,trial,chrt)
            if rho_data.empty:
                print("Nothing in Rho!")
                continue
            visit_info = querying.get_visit_info(pd,cnxn,trial)
            if trial.cohort[0] == chrt:
                result = create_result_df.create_result_df(pd,trial,chrt,rho_data,lv_data,visit_info)
                result = create_result_df.remove_exceptions(np,result,trial,chrt)
                print("Finished {} Cohort {}".format(trial.studynum,chrt))
            else:
                result_to_add = create_result_df.create_result_df(pd,trial,chrt,rho_data,lv_data,visit_info)
                result_to_add = create_result_df.remove_exceptions(np,result_to_add,trial,chrt)
                result = pd.concat([result,result_to_add])
                print("Finished {} Cohort {}".format(trial.studynum,chrt))

    else:
        lv_data = querying.get_lv_data(pd,cnxn,trial,None)
        rho_data = querying.get_rho_data(pd,cnxn,trial,None)
        visit_info = querying.get_visit_info(pd,cnxn,trial)
        result = create_result_df.create_result_df(pd,trial,chrt,rho_data,lv_data,visit_info)
        result = create_result_df.remove_exceptions(np,result,trial,None)

    output = pd.concat([output,result])

cnxn.close()

output

Starting ITN080AI Cohort A


c:\Users\lserizawa\Desktop\Python Environ\SRC table\querying.py:9: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  output_df = pd.read_sql(lv_query,cnxn)
c:\Users\lserizawa\Desktop\Python Environ\SRC table\querying.py:113: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  output_df = pd.read_sql(rho_query,cnxn)
c:\Users\lserizawa\Desktop\Python Environ\SRC table\querying.py:152: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  output_df = pd.read_sql(visit_query,cnxn)


Finished ITN080AI Cohort A
Starting ITN080AI Cohort B


c:\Users\lserizawa\Desktop\Python Environ\SRC table\querying.py:9: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  output_df = pd.read_sql(lv_query,cnxn)
c:\Users\lserizawa\Desktop\Python Environ\SRC table\querying.py:113: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  output_df = pd.read_sql(rho_query,cnxn)
c:\Users\lserizawa\Desktop\Python Environ\SRC table\querying.py:152: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  output_df = pd.read_sql(visit_query,cnxn)


Finished ITN080AI Cohort B


...

c:\Users\lserizawa\Desktop\Python Environ\SRC table\querying.py:113: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  output_df = pd.read_sql(rho_query,cnxn)
c:\Users\lserizawa\Desktop\Python Environ\SRC table\querying.py:152: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  output_df = pd.read_sql(visit_query,cnxn)


Starting ITN089ST Cohort Donor
Nothing in Rho!
Starting ITN089ST Cohort Recipient


c:\Users\lserizawa\Desktop\Python Environ\SRC table\querying.py:9: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  output_df = pd.read_sql(lv_query,cnxn)
c:\Users\lserizawa\Desktop\Python Environ\SRC table\querying.py:113: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  output_df = pd.read_sql(rho_query,cnxn)
c:\Users\lserizawa\Desktop\Python Environ\SRC table\querying.py:9: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  output_df = pd.read_sql(lv_query,cnxn)
c:\Users\lserizawa\Desktop\Python Environ\SRC table\querying.py:113: U

Nothing in Rho!
Starting ITN089ST Cohort Ancillary
Nothing in Rho!


,Study,Cohort,Visit Number,Visit Ordinal,DaysPostScreening,Sample Type,Number at least 1 collected,Number of recorded visits,Percent
0,ITN080AI,A,0A,15,0.0,Serum Clot,17,17,100.0
1,ITN080AI,A,0A,15,0.0,PBMC,17,17,100.0
2,ITN080AI,A,0A,15,0.0,Whole Blood\nTranscriptomics,16,17,94.0
3,ITN080AI,A,0A,15,0.0,Whole Blood\nEpigenetic,17,17,100.0
4,ITN080AI,A,0A,15,0.0,Urine Supernatant,17,17,100.0
...,...,...,...,...,...,...,...,...,...
75,ITN077AI,None,PT,175,NaN,Whole Blood\nGene Expression,0,1,0.0
76,ITN077AI,None,PT,175,NaN,CSF Transfix,0,1,0.0
77,ITN077AI,None,PT,175,NaN,CSF Super,0,1,0.0
78,ITN077AI,None,PT,175,NaN,CSF Pellet,0,1,0.0


In [3]:
output.Study.unique()

array(['ITN080AI', 'ITN086AI', 'ITN084AD', 'ITN077AI'], dtype=object)